# Importing libraries

In [525]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import pandas as pd
import re

# Webscraping

List of promissing regions:

- new york
- los angeles
- 

#the function has to get
primeira parte:
- get event name
- get event date
- get event local
- get event website


segunda parte:
- Get adress
- get lat long

In [839]:
def extract_events(months,cities,country,year=2020):
    '''
    This function shows all the eventes listed on www.residentadvisor.net.
    It returns a dataframe listing event's date, name, place, city, country and link.
    User must insert dates to be searched as a list, the country and a list of cities name,
    although it has default values inserted.
    
    months must be a list with the number of the months, like: [01, 02, 03, 04, ... 11, 12]
    
    '''
    df = pd.DataFrame()
    
    print('Extract_events progress:')
    for city in tqdm(cities):

        for month in months:

            url = f'https://www.residentadvisor.net/events/{country}/{city}/month/2020-'+ f'{month}' +'-01'
            
            response = requests.get(url)
            soup = BeautifulSoup(response.content)


            # the body of the html where our info is, is located at  <div id="event-listing" class="fl col4">
            minisoup = soup.find_all('div',attrs={'id':'event-listing'})
            
            #we first check if there is any events on the searched month... if so, we analyse the soup
            if len(minisoup) != 0:
                minisoup = minisoup[0]
                #now we analyse line by line: <article class="event-item pick clearfix  tickets-bkg-logo small-item" itemscope=""...>
                html_chunks = minisoup.find_all('article')
                
                for html_chunk in html_chunks:

                    link = 'https://www.residentadvisor.net' + html_chunk.find_all('a')[0]['href'].split('#tickets')[0]
                    event_date = html_chunk.find_all('time')[0].text.split('T')[0]
                    event_name = html_chunk.find_all('h1')[0].text.split('at')[0].strip()
                    event_place = html_chunk.find_all('h1')[0].text.split('at')[1].strip()

                    mini_df = pd.DataFrame({'date' : [event_date],
                                        'name' : [event_name],
                                        'place': [event_place],
                                            'city' : [city],
                                            'country' : [country],
                                        'link': [link]})
                    df = pd.concat([df, mini_df])

    print('------------- Process concluded! ------------')
    df = df.reset_index(drop=True)
    return df

In [394]:
def get_event_info(links):
    '''
    This function checks links of events and gets, when existing, info about:
    Date, Adress, Minium Age, Promoters, line-up, event description and Cost'
    All info is returned as a dataframe
    '''
    
    df = pd.DataFrame()

    for link in tqdm(links):

        temp_dict = {}

        url = link
        response = requests.get(url)
        soup = BeautifulSoup(response.content)

        # all we need is inside <ul class="clearfix">, the first one
        chunk = soup.find_all('ul',attrs={'class':'clearfix'})[0]
        infos = [piece.text for piece in chunk.find_all('li')]

        for info in infos:
            key = info.split('/')[0].strip()
            value = info.split('/')[1]
            temp_dict[key] = value

        temp_dict['lineup'] = soup.find_all('p',attrs={'class':'lineup'})[0].text.replace('\n',', ')
        temp_dict['description'] = soup.find_all('p')[1].text.strip().replace('\n',' / ')
        temp_dict['Date'] = temp_dict['Date'].split('2020')[-1]

        mini_df = pd.DataFrame(temp_dict,index=[0])
        df = pd.concat([df, mini_df])
    
    # Renamin columns
    df.columns = ['event_time', 'Address', 'Minimum age', 'Promoters', 'lineup', 'description', 'Cost']
    return df.reset_index(drop=True)

# API: locationiq.com

GET https://us1.locationiq.com/v1/search.php?key=YOUR_PRIVATE_TOKEN&q=SEARCH_STRING&format=json

In [777]:
def get_coordinates_us_old(search_string, token='7c1406efadcbfe', sleeptime=0.5):
    '''
    This function receives an address and returns latitude and longitude coordinates as a tuple.
    Sleep is needed to avoid excess requests to API, since is limited with current key.
    '''
    
    import time
    time.sleep(sleeptime)
    
    
    if search_string == 'undefined':
        return np.nan
    
    url = f'https://us1.locationiq.com/v1/search.php?key={token}&q={search_string}&format=json'
    response = requests.get(url)

    #if response is a json
    try:
        coordinates = (response.json()[0]['lat'],response.json()[0]['lon'])
        return coordinates

    #if response is 'xml or html'
    except:
        soup = BeautifulSoup(response.content,'xml')

        if len(soup.find_all('searchresults')) !=0:
            # its a normal xml
            values = soup.find_all('place')[0]['boundingbox'].split(',')
            coordinates = (values[0],values[2])
            return coordinates

        else:
            # its a normal html
            soup = BeautifulSoup(response.content)
            coordinates = [piece for piece in soup.find_all('p')[0].text.split(',') if (piece.startswith('"lat"') == True or piece.startswith('"lon"') == True)]
            return (coordinates[0].split('"')[3], coordinates[1].split('"')[3])
    


In [796]:
def get_coordinates_us(search_string, token='7c1406efadcbfe', sleeptime=0.75):
    '''
    This function receives an address and returns latitude and longitude coordinates as a tuple.
    Sleep is needed to avoid excess requests to API, since is limited with current key.
    Going too fast makes the API return not what is expected, which ends up in error
    '''
    
    import time
    time.sleep(sleeptime)
    
    
    if search_string == 'undefined':
        return np.nan
    
    url = f'https://us1.locationiq.com/v1/search.php?key={token}&q={search_string}&format=json'
    response = requests.get(url)

    #if response is a json
    try:
        coordinates = (response.json()[0]['lat'],response.json()[0]['lon'])
        return coordinates

    #if response is 'xml or html'
    except:
        soup = BeautifulSoup(response.content,'xml')
        values = soup.find_all('place')[0]['boundingbox'].split(',')
        coordinates = (values[0],values[2])
        return coordinates

In [578]:
#testing
get_coordinates_us('1809 Minor Ave #10')

('47.6168713', '-122.3311299')

In [522]:
#testing
get_coordinates_us('428 S Hewitt St, Los Angeles')

('34.04272815', '-118.234942180885')

# API: https://www.hikingproject.com/data

In [853]:
def get_hiking_parks(coord, address_id, key='200715322-84f39d4737b94d74882debf302722ab5', maxD = 20):
    '''
    This function receives latitude and longitude coordinates and returns a dataframe containing information
    regarding parks near the location suited for camping or hinking.
    If there are no parks near the coordinates given, an empty dataframe 
    
    maxD - Max distance, in miles, from lat, lon. Default: 30. Max: 200.
    '''
    
    import numpy as np  
    
    url = f'https://www.hikingproject.com/data/get-campgrounds?lat={coord[0]}&lon={coord[1]}&maxDistance={maxD}&key={key}'
    response = requests.request('get',url)
    success = response.json()['success']
    campgrounds = response.json()['campgrounds']
    
    if success == 1:
        df = pd.DataFrame(campgrounds)
        df['address'] = address_id
#         df = df.set_index(keys='event near').reset_index()
        return df
    
    else:
        empty_dict = {"address":address_id,
                      "id": np.nan,
                      "name":  np.nan,
                      "isBookable": np.nan,
                      "isCampground": np.nan,
                      "location": np.nan,
                      "latitude": np.nan,
                      "longitude": np.nan,
                      "url": np.nan,
                      "imgUrl": np.nan,
                      "numCampsites": np.nan}
        
        return pd.DataFrame(empty_dict, index=[0])

In [ ]:
#testing:
coordinates = ('34.04272815', '-118.234942180885')
get_hiking_parks(coord=coordinates,address_id='FiestaFiesta')

# Testing

In [ ]:
dates = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
list_of_cities = ['seattle', 'houston','newyork','detroit','losangeles','orlando','sanfrancisco','sandiego']

# getting events
events = extract_events(months=dates[3:],country='us',cities=list_of_cities)

In [445]:
# getting events info
events_info = get_event_info(links = events.link)

In [817]:
#cleaning Address column from '\xa0 United States of America'

df_infos = events_info.copy()
df_infos.loc[:,'Address'] = df_infos.loc[:,'Address'].apply(lambda x: x.split('\xa0')[0])

#second, every address starts with a number, so anything before this number is the name of the place. 
#Also, in case no number is found, the information will be rewritten as 'undefined'
#one last thing, the numbers must be followed by a letter, which avoid getting pieces of names that contain numbers (like 'bar 666')
pattern = '\d{3}\d? [a-zA-Z].*'
df_infos.loc[:,'Address'] = df_infos.loc[:,'Address'].apply(lambda x: re.findall(pattern,x)[0] if len(re.findall(pattern,x)) !=0 else 'undefined')

#extracting now coordinates. We need to use unique addresses to avoid double requesting the API
to_request = df_infos.Address.unique()
temp_dict = {}

for request in tqdm(to_request):
    temp_dict[request] = get_coordinates_us(request)

df_infos['coordinates'] = df_infos.loc[:,'Address'].apply(lambda x: temp_dict[x])
event_list = pd.concat([events,df_infos],axis=1)

In [843]:
event_list.loc[:,['Address','coordinates']].dropna()

,Address,coordinates
0,"1809 Minor Ave #10, Seattle, Washington 98101","(47.6168713, -122.3311299)"
1,"1809 Minor Ave #10, Seattle, Washington 98101","(47.6168713, -122.3311299)"
2,"1426 Broadway, Seattle, WA.","(47.613746, -122.3203363)"
4,"1426 Broadway, Seattle, WA.","(47.613746, -122.3203363)"
6,"925 East Pike Street; Seattle, WA 98122; Unite...","(47.613959, -122.319574)"
...,...,...
577,"2223 El Cajon Boulevard, San Diego, CA 92104","(32.7552354, -117.131821)"
578,"2223 El Cajon Boulevard, San Diego, CA 92104","(32.7552354, -117.131821)"
579,"2223 El Cajon Boulevard, San Diego, CA 92104","(32.7552354, -117.131821)"
580,"868 Fourth Ave, San Diego, CA 92101, Estados U...","(32.597701, -117.067971)"


In [864]:
# creating dataframe of parks. This dataframe will be connected by an address ID, since parties and
# places can have the same names but different addresses

df_temp = event_list.loc[:,['Address','coordinates']].dropna()

dfparks = pd.DataFrame()

for i in tqdm(range(df_temp.shape[0])):
    coord = df_temp.loc[:,'coordinates'].iloc[i]
    address_id = df_temp.loc[:,'Address'].iloc[i]
    mini_df = get_hiking_parks(coord, address_id,maxD=30)
    dfparks = pd.concat([dfparks, mini_df])


In [865]:
dfparks

,address,id,name,isBookable,isCampground,location,latitude,longitude,url,imgUrl,numCampsites
0,"599 Johnson Ave, Brooklyn, NY 11237",115196040.0,Camp Gateway- Brooklyn Ny,True,True,"East New York, New York",40.5958,-73.8858,https://www.rei.com/campgrounds/camp/115196040...,https://cdn.recreation.gov/public/images/69035...,45.0
1,"599 Johnson Ave, Brooklyn, NY 11237",115197387.0,Camp Gateway - Staten Island,True,True,"Bensonhurst, New York",40.6036,-74.0588,https://www.rei.com/campgrounds/camp/115197387...,https://cdn.recreation.gov/public/2018/08/16/2...,7.0
2,"599 Johnson Ave, Brooklyn, NY 11237",115197386.0,Camp Gateway - Sandy Hook,True,True,"Highlands, New Jersey",40.4492,-73.9961,https://www.rei.com/campgrounds/camp/115197386...,https://cdn.recreation.gov/public/2018/08/02/1...,20.0
3,"599 Johnson Ave, Brooklyn, NY 11237",115208172.0,Theodore Roosevelt Home,False,False,"Oyster Bay Cove, New York",40.8855,-73.5012,https://www.rei.com/campgrounds/camp/115208172...,https://www.rei.com/assets/camp/images/campgro...,0.0
0,"1271 Myrtle Ave; Brooklyn, NY 11221; United St...",115196040.0,Camp Gateway- Brooklyn Ny,True,True,"East New York, New York",40.5958,-73.8858,https://www.rei.com/campgrounds/camp/115196040...,https://cdn.recreation.gov/public/images/69035...,45.0
...,...,...,...,...,...,...,...,...,...,...,...
0,"2223 El Cajon Boulevard, San Diego, CA 92104",115205702.0,Descanso Rd,False,False,"Alpine, California",32.8342,-116.7427,https://www.rei.com/campgrounds/camp/115205702...,https://www.rei.com/assets/camp/images/campgro...,0.0
0,"2223 El Cajon Boulevard, San Diego, CA 92104",115205702.0,Descanso Rd,False,False,"Alpine, California",32.8342,-116.7427,https://www.rei.com/campgrounds/camp/115205702...,https://www.rei.com/assets/camp/images/campgro...,0.0
0,"868 Fourth Ave, San Diego, CA 92101, Estados U...",115205702.0,Descanso Rd,False,False,"Alpine, California",32.8342,-116.7427,https://www.rei.com/campgrounds/camp/115205702...,https://www.rei.com/assets/camp/images/campgro...,0.0
1,"868 Fourth Ave, San Diego, CA 92101, Estados U...",115201512.0,Pine Creek/Hauser Wilderness Area,False,False,"Descanso, California",32.7827,-116.6489,https://www.rei.com/campgrounds/camp/115201512...,https://www.rei.com/assets/camp/images/campgro...,0.0
